In [28]:
import pandas as pd

import nltk
# nltk.download('stopwords')
# nltk.download('punkt_tab')

from nltk.corpus import stopwords
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

import json


# Data Modeling 
# pick two topics to compare
# develop a way to identify which topic a post is talking about 
# check occurences of different topics of conversation over time
#   could these different conversation topics appear during a game period more often?


In [27]:
ps = PorterStemmer()

def text2tokens(text):
	stop_words = set(stopwords.words('english'))
	text = str(text)
	text = text.lower()
	textList = word_tokenize(text)
	textList = [word for word in textList if word not in stop_words and len(word) >= 3]

	textList = [ps.stem(word) for word in textList]
	return textList
    
def gen_bow(df, column):
	df['tokens'] = df[column].apply(text2tokens)
	dct = Dictionary(df['tokens'])
	dct.filter_extremes(no_below=5, no_above=0.5)
	df['bow'] = df['tokens'].apply(dct.doc2bow)
	worddict = {}

	for i in range(len(df)):
		for token in df.at[i,'bow']:
			if token in list(worddict.keys()):
				worddict[token] += 1
			else: 
				worddict[token] = 1

	df.drop('tokens', axis=1, inplace=True)	
	return df, worddict

posNegWords = pd.read_excel('posNegList.xlsx').dropna()
# print(posNegWords.dtypes)
posWords = [ps.stem(word) for word in posNegWords['Positive Sense Word List'].to_list()]
negWords = [ps.stem(word) for word in posNegWords['Negative Sense Word List'].to_list()]



# display(posWords)

def get_sentiment(list):
	sentiment = 0
	
	for word in list:
		if word in posWords:
			sentiment += 1
		elif word in negWords:
			sentiment -= 1
	
	return sentiment
			
or_df = pd.read_csv('hw2_step1_or_posts.csv')

or_df, or_dict = gen_bow(or_df, 'text')
display(or_df)
or_df['tokens'] = or_df['text'].apply(text2tokens)
or_df['sentiment'] = or_df['tokens'].apply(get_sentiment)

display(or_df)


,text,time_of_post,game,bow
0,first career complete game no-hitter for Means!,2021-05-05,"['401227501', '401227486']","[(0, 1), (1, 1), (2, 1), (3, 1)]"
1,#FIRST NO HITTER IN 30 YEARS LETS GOOOOOOOO,2021-05-05,"['401227501', '401227486']","[(1, 1), (4, 1), (5, 1), (6, 1), (7, 1)]"
2,I can't believe the son of a bitch actually di...,2021-05-05,"['401227501', '401227486']","[(5, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12..."
3,EVERYBODY GET THE FUCK IN HERE PAT HIT HIS FIR...,2021-05-05,"['401227501', '401227486']","[(1, 1), (3, 1), (5, 1), (16, 1), (17, 1), (18..."
4,WE'VE SUCKED FOR SO LONG HOW DO I TAKE THIS IN...,2021-05-05,"['401227501', '401227486']","[(22, 1), (23, 1), (24, 1), (25, 1), (26, 1)]"
...,...,...,...,...
828,More fun watch too!,2019-09-14,['401076950'],"[(39, 1), (293, 1)]"
829,What’s sad is that we were actually trying to ...,2019-09-14,['401076950'],"[(7, 1), (8, 1), (92, 1), (105, 1), (266, 1)]"
830,WOOHOO!,2019-09-14,['401076950'],[]
831,"Win or lose, I love the Birds. Since 1954. Al...",2019-09-14,['401076950'],"[(45, 1), (53, 1), (64, 1), (96, 1), (144, 1),..."


,text,time_of_post,game,bow,tokens,sentiment
0,first career complete game no-hitter for Means!,2021-05-05,"['401227501', '401227486']","[(0, 1), (1, 1), (2, 1), (3, 1)]","[first, career, complet, game, no-hitt, mean]",5
1,#FIRST NO HITTER IN 30 YEARS LETS GOOOOOOOO,2021-05-05,"['401227501', '401227486']","[(1, 1), (4, 1), (5, 1), (6, 1), (7, 1)]","[first, hitter, year, let, goooooooo]",1
2,I can't believe the son of a bitch actually di...,2021-05-05,"['401227501', '401227486']","[(5, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12...","[n't, believ, son, bitch, actual, never, thoug...",1
3,EVERYBODY GET THE FUCK IN HERE PAT HIT HIS FIR...,2021-05-05,"['401227501', '401227486']","[(1, 1), (3, 1), (5, 1), (16, 1), (17, 1), (18...","[everybodi, get, fuck, pat, hit, first, home, ...",4
4,WE'VE SUCKED FOR SO LONG HOW DO I TAKE THIS IN...,2021-05-05,"['401227501', '401227486']","[(22, 1), (23, 1), (24, 1), (25, 1), (26, 1)]","['ve, suck, long, take, without, tear]",-1
...,...,...,...,...,...,...
828,More fun watch too!,2019-09-14,['401076950'],"[(39, 1), (293, 1)]","[fun, watch]",2
829,What’s sad is that we were actually trying to ...,2019-09-14,['401076950'],"[(7, 1), (8, 1), (92, 1), (105, 1), (266, 1)]","[sad, actual, tri, good, last, year]",1
830,WOOHOO!,2019-09-14,['401076950'],[],[woohoo],0
831,"Win or lose, I love the Birds. Since 1954. Al...",2019-09-14,['401076950'],"[(45, 1), (53, 1), (64, 1), (96, 1), (144, 1),...","[win, lose, love, bird, sinc, 1954., upvot]",2
